In [ ]:
import datetime
import matplotlib.pyplot as plt
import sqlite3

# Connect to SQLite database (create one if it doesn't exist)
conn = sqlite3.connect('fitness_tracker.db')
cursor = conn.cursor()

# Create tables if they don't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        user_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        age INTEGER NOT NULL
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS weight_logs (
        log_id INTEGER PRIMARY KEY,
        user_id INTEGER NOT NULL,
        weight REAL NOT NULL,
        log_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (user_id) REFERENCES users(user_id)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS workout_logs (
        log_id INTEGER PRIMARY KEY,
        user_id INTEGER NOT NULL,
        workout_type TEXT NOT NULL,
        duration REAL NOT NULL,
        calories_burned INTEGER NOT NULL,
        log_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (user_id) REFERENCES users(user_id)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS meal_logs (
        log_id INTEGER PRIMARY KEY,
        user_id INTEGER NOT NULL,
        meal_type TEXT NOT NULL,
        calories_consumed INTEGER NOT NULL,
        log_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (user_id) REFERENCES users(user_id)
    )
''')

conn.commit()

def log_weight(user_id, weight):
    try:
        cursor.execute('''
            INSERT INTO weight_logs (user_id, weight) VALUES (?, ?)
        ''', (user_id, weight))
        conn.commit()
        print(f"Weight of {weight} kg logged for user {user_id}.")
    except ValueError:
        print("Invalid input for weight. Please enter a valid number.")

def log_workout(user_id, workouts):
    try:
        for workout, duration, calories_burned in workouts:
            cursor.execute('''
                INSERT INTO workout_logs (user_id, workout_type, duration, calories_burned) VALUES (?, ?, ?, ?)
            ''', (user_id, workout, duration, calories_burned))
        conn.commit()
        print(f"Logged workout details for user {user_id}.")
    except ValueError:
        print("Invalid input for workout details. Please enter valid numbers.")

def log_meal(user_id, meals):
    try:
        for meal, calories in meals:
            cursor.execute('''
                INSERT INTO meal_logs (user_id, meal_type, calories_consumed) VALUES (?, ?, ?)
            ''', (user_id, meal, calories))
        conn.commit()
        print(f"Logged meal details for user {user_id}.")
    except ValueError:
        print("Invalid input for meal details. Please enter valid numbers.")

def display_logs():
    # Display logs for all users
    cursor.execute('SELECT * FROM users')
    users = cursor.fetchall()

    for user in users:
        user_id = user[0]
        user_name = user[1]
        print(f"\nLogs for User {user_id} ({user_name}):")

        # Display weight log
        cursor.execute('SELECT * FROM weight_logs WHERE user_id = ?', (user_id,))
        weight_logs = cursor.fetchall()
        if weight_logs:
            print("Weight Log:")
            for log in weight_logs:
                print(f"{log[2]} kg on {log[3]}")

        # Display workout log
        cursor.execute('SELECT * FROM workout_logs WHERE user_id = ?', (user_id,))
        workout_logs = cursor.fetchall()
        if workout_logs:
            print("\nWorkout Log:")
            for log in workout_logs:
                print(f"{log[2]} for {log[3]} minutes, burning {log[4]} calories on {log[5]}")

        # Display meal log
        cursor.execute('SELECT * FROM meal_logs WHERE user_id = ?', (user_id,))
        meal_logs = cursor.fetchall()
        if meal_logs:
            print("\nMeal Log:")
            for log in meal_logs:
                print(f"{log[2]} ({log[3]} calories) on {log[4]}")

        # Analyze calories consumed vs. burned
        analyze_calories_consumed_vs_burned(user_id)

def analyze_calories_consumed_vs_burned(user_id):
    try:
        # Get total calories burned from all workouts for the user
        cursor.execute('SELECT SUM(calories_burned) FROM workout_logs WHERE user_id = ?', (user_id,))
        total_calories_burned = cursor.fetchone()[0] or 0

        # Get total calories consumed from all meals for the user
        cursor.execute('SELECT SUM(calories_consumed) FROM meal_logs WHERE user_id = ?', (user_id,))
        total_calories_consumed = cursor.fetchone()[0] or 0

        if total_calories_burned == 0 and total_calories_consumed == 0:
            print(f"No workout or meal transactions recorded yet for user {user_id}.")
            return

        # Plot the data as a pie chart
        labels = ['Calories Burned', 'Calories Consumed']
        sizes = [total_calories_burned, total_calories_consumed]

        plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
        plt.axis('equal')  # Equal aspect ratio ensures the pie chart is circular.
        plt.title(f'Calories Burned vs Consumed for User {user_id}')
        plt.show()

    except Exception as e:
        print(f"An error occurred while analyzing calories for user {user_id}: {e}")

# ...

# Get the number of users with exception handling
while True:
    try:
        num_users = int(input("Enter the number of users: "))
        if num_users <= 0:
            raise ValueError("Number of users should be a positive integer.")
        break
    except ValueError:
        print("Invalid input for the number of users. Please enter a valid number.")

# Loop to enter information for each user
for i in range(num_users):
    print(f"\nUser {i+1} Information:")
    user_name = input("Enter user name: ")
    while True:
        user_age_input = input("Enter user age: ")
        try:
            user_age = int(user_age_input)
            if user_age <= 0:
                raise ValueError("Age should be a positive integer.")
            break
        except ValueError:
            print("Invalid input for age. Please enter a valid number.")

    # Insert user into the database
    cursor.execute('INSERT INTO users (name, age) VALUES (?, ?)', (user_name, user_age))
    user_id = cursor.lastrowid

    # Log weight
    while True:
        weight_input = input(f"\nEnter weight for user {user_id} (in kilograms): ")
        try:
            weight = float(weight_input)
            break
        except ValueError:
            print("Invalid input for weight. Please enter a valid number.")
    log_weight(user_id, weight)

    # Log workouts
    workouts = []
    while True:
        num_workouts_input = input(f"How many workouts would you like to log for user {user_id}? ")
        try:
            num_workouts = int(num_workouts_input)
            if num_workouts < 0:
                raise ValueError("Number of workouts should be a non-negative integer.")
            break
        except ValueError:
            print("Invalid input for the number of workouts. Please enter a valid number.")

    for j in range(num_workouts):
        workout_type = input(f"Enter workout {j+1} type: ")
        while True:
            duration_input = input(f"Enter the duration (in minutes) for workout {j+1}: ")
            try:
                duration = float(duration_input)
                if duration < 0:
                    raise ValueError("Duration should be a non-negative number.")
                break
            except ValueError:
                print("Invalid input for duration. Please enter a valid number.")
        while True:
            calories_burned_input = input(f"How many calories burned for workout {j+1}: ")
            try:
                calories_burned = int(calories_burned_input)
                if calories_burned < 0:
                    raise ValueError("Calories burned should be a non-negative number.")
                break
            except ValueError:
                print("Invalid input for calories burned. Please enter a valid number.")
        workouts.append((workout_type, duration, calories_burned))
    log_workout(user_id, workouts)

    # Log meals
    meals = []
    while True:
        num_meals_input = input(f"How many meals would you like to log for user {user_id}? ")
        try:
            num_meals = int(num_meals_input)
            if num_meals < 0:
                raise ValueError("Number of meals should be a non-negative integer.")
            break
        except ValueError:
            print("Invalid input for the number of meals. Please enter a valid number.")

    for k in range(num_meals):
        meal_type = input(f"Enter meal {k+1} description: ")
        while True:
            calories_consumed_input = input(f"Enter the calorie count for meal {k+1}: ")
            try:
                calories_consumed = int(calories_consumed_input)
                if calories_consumed < 0:
                    raise ValueError("Calories consumed should be a non-negative number.")
                break
            except ValueError:
                print("Invalid input for calorie count. Please enter a valid number.")
        meals.append((meal_type, calories_consumed))
    log_meal(user_id, meals)

# Display logs for all users (outside the loop)
display_logs()

# Close the database connection when the program exits
conn.close()
